# UNESCO项目数据筛选与处理

本笔记本用于筛选UNESCO项目数据中的关键字段，并将结果保存到新表中。

## 1. 导入必要的库

In [ ]:
import pandas as pd
import numpy as np
import json

## 2. 设置文件路径

In [ ]:
# 输入文件路径
input_file = 'UNESCO-projects-prj001.csv'

# 输出文件路径
output_file = 'UNESCO_projects_filtered.csv'

## 3. 读取原始数据

In [ ]:
# 读取CSV文件
try:
    # 读取整个CSV文件
    df = pd.read_csv(input_file, encoding='utf-8')
    print(f'成功读取数据，共 {len(df)} 条记录')
    print('数据字段：')
    for i, col in enumerate(df.columns):
        print(f'{i+1}. {col}')
except Exception as e:
    print(f'读取文件时出错: {e}')
    # 尝试使用不同的编码
    try:
        df = pd.read_csv(input_file, encoding='latin-1')
        print(f'使用latin-1编码成功读取数据，共 {len(df)} 条记录')
    except Exception as e:
        print(f'使用latin-1编码读取也失败: {e}')

## 4. 筛选所需字段

In [ ]:
# 定义需要筛选的字段
required_fields = [
    'Project',              # 项目名称
    'Description',         # 项目描述
    'Outcome EN',          # 预期成果
    'Output EN',           # 具体产出
    'Sector',              # 部门
    'Beneficiary',         # 受益方
    'Donor',               # 捐赠者
    'Beneficiary Type',    # 受益方类型
    'Regional Group',      # 地区组（Region信息）
    'Start Date',          # 开始日期
    'End Date',            # 结束日期
    'Coordinates'          # 坐标
]

# 检查字段是否存在
missing_fields = [field for field in required_fields if field not in df.columns]
if missing_fields:
    print(f'警告：以下字段在数据中不存在：{missing_fields}')
else:
    print('所有所需字段都存在于数据中')

# 创建筛选后的数据框
# 只选择存在的字段
existing_required_fields = [field for field in required_fields if field in df.columns]
filtered_df = df[existing_required_fields].copy()
print(f'筛选后的数据维度: {filtered_df.shape}')
print('筛选后的字段:')
for i, col in enumerate(filtered_df.columns):
    print(f'{i+1}. {col}')

## 5. 数据清洗和转换

In [ ]:
# 5.1 处理缺失值
print('缺失值统计:')
missing_data = filtered_df.isnull().sum()
missing_percentage = (missing_data / len(filtered_df) * 100).round(2)
missing_info = pd.DataFrame({
    '缺失数量': missing_data,
    '缺失百分比(%)': missing_percentage
})
missing_info = missing_info[missing_info['缺失数量'] > 0]
print(missing_info)

# 5.2 处理日期字段
date_fields = ['Start Date', 'End Date']
for field in date_fields:
    if field in filtered_df.columns:
        try:
            filtered_df[field] = pd.to_datetime(filtered_df[field], errors='coerce')
            print(f'已将 {field} 转换为日期格式')
        except Exception as e:
            print(f'转换 {field} 到日期格式时出错: {e}')

# 5.3 处理文本字段，去除多余空格
text_fields = ['Project', 'Description', 'Outcome EN', 'Output EN', 'Sector', 'Beneficiary', 'Donor', 'Beneficiary Type', 'Regional Group']
for field in text_fields:
    if field in filtered_df.columns:
        # 处理字符串类型的字段
        mask = filtered_df[field].apply(lambda x: isinstance(x, str))
        filtered_df.loc[mask, field] = filtered_df.loc[mask, field].str.strip()
        print(f'已处理 {field} 的文本格式')

# 5.4 检查并处理重复行
duplicate_rows = filtered_df.duplicated().sum()
if duplicate_rows > 0:
    print(f'发现 {duplicate_rows} 条重复记录，将被移除')
    filtered_df = filtered_df.drop_duplicates()
    print(f'去重后的数据维度: {filtered_df.shape}')
else:
    print('没有发现重复记录')

## 6. 数据质量检查

In [ ]:
# 显示数据基本信息
print('筛选后数据的基本信息:')
filtered_df.info()

# 显示前5行数据样例
print('数据样例 (前5行):')
display(filtered_df.head())

# 统计Sector、Beneficiary Type和Regional Group的分布
for field in ['Sector', 'Beneficiary Type', 'Regional Group']:
    if field in filtered_df.columns:
        print(f'{field} 分布:')
        print(filtered_df[field].value_counts().head(10))

## 7. 保存筛选后的数据

In [ ]:
# 保存筛选后的数据到新的CSV文件
try:
    filtered_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f'数据已成功保存到 {output_file}')
    print(f'保存了 {len(filtered_df)} 条记录')
except Exception as e:
    print(f'保存文件时出错: {e}')
    # 尝试使用不同的编码
    try:
        filtered_df.to_csv(output_file, index=False, encoding='latin-1')
        print(f'使用latin-1编码成功保存数据到 {output_file}')
    except Exception as e:
        print(f'使用latin-1编码保存也失败: {e}')

## 8. 总结

In [ ]:
print('===== 数据筛选任务完成 =====')
print(f'- 原始数据记录数: {len(df)}')
print(f'- 筛选后数据记录数: {len(filtered_df)}')
print(f'- 筛选的字段数: {len(filtered_df.columns)}')
print(f'- 输出文件: {output_file}')
print('筛选的字段包括:')
for field in filtered_df.columns:
    print(f'  - {field}')